In [12]:
import config
import helpers

import pandas as pd
from fredapi import Fred
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
%matplotlib inline

#Plotly Dash components 
import jupyterlab_dash
import dash
import dash_core_components as dcc
import dash_html_components as html
external_stylesheets = ['https://codepen.io/gullpavon1/pen/bGVWQEr.css']

#Connect to Federal Reserve Economic Data
fred = Fred(api_key=config.fred_api_code)

print('Import Complete.')
#Search for FRED data
#fred.search('mortgage').T


Import Complete.


In [2]:
#Dash App
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=False)

NameError: name 'dash' is not defined

In [3]:

MORTGAGE30US = fred.get_series_all_releases('MORTGAGE30US')
MORTGAGE30US = MORTGAGE30US[MORTGAGE30US.date >= '2020-01-01']



NameError: name 'fred' is not defined

In [4]:
#30 Yr Mortgage Rates
MORTGAGE30US.plot.line(x='date', y='value')
plt.title ('30 Yr Mortgage Rates')
plt.xlabel ('Date')
plt.ylabel ('Rate')


NameError: name 'MORTGAGE30US' is not defined

In [5]:

#Unemployment Rate
#For Unemployment number use: UNEMPLOY
UNRATE = fred.get_series_all_releases('UNRATE')
UNRATE = UNRATE[UNRATE.date >= '2019-01-01']

UNRATE.plot.line(x='date', y='value')
plt.title ('Unemployment Rate')
plt.xlabel ('Date')
plt.ylabel ('Rate')

NameError: name 'fred' is not defined

In [6]:


#Fed Funds Rate
FEDFUNDS = fred.get_series_all_releases('FEDFUNDS')
FEDFUNDS = FEDFUNDS[FEDFUNDS.date >= '2019-01-01']

FEDFUNDS.plot.line(x='date', y='value')
plt.title ('Federal Funds Rate')
plt.xlabel ('Date')
plt.ylabel ('Rate')

NameError: name 'fred' is not defined

In [7]:

#10-2 Treasury Yield Spread
T10Y2Y = fred.get_series_all_releases('T10Y2Y')
T10Y2Y = T10Y2Y[T10Y2Y.date >= '2019-01-01']
T10Y2Y.dropna(inplace= True)

T10Y2Y.plot.line(x='date', y='value')
plt.title ('10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity ')
plt.xlabel ('Date')
plt.ylabel ('Rate')
plt.axhline(0, 0, 1, label='0')

NameError: name 'fred' is not defined

In [8]:
#10-2 Treasury Yield Spread
T10YIE = fred.get_series_all_releases('T10YIE')
T10YIE = T10YIE[T10YIE.date >= '2020-01-01']
T10YIE.dropna(inplace= True)


T10YIE.plot.line(x='date', y='value')
plt.title ('10-Year Breakeven Inflation Rate')
plt.xlabel ('Date')
plt.ylabel ('Rate')
plt.axhline(0, 0, 1, label='0')


NameError: name 'fred' is not defined

In [9]:

CSUSHPINSA = fred.get_series_all_releases('CSUSHPINSA')
CSUSHPINSA = CSUSHPINSA[CSUSHPINSA.date >= '2019-01-01']
CSUSHPINSA.dropna(inplace= True)
CSUSHPINSA.plot.line(x='date', y='value')

plt.title ('S&P/Case-Shiller U.S. National Home Price Index')
plt.xlabel ('Date')
plt.ylabel ('Rate')
plt.axhline(0, 0, 1, label='0')

NameError: name 'fred' is not defined

In [10]:

IOER = fred.get_series_all_releases('IOER')
IOER = IOER[IOER.date >= '2019-01-01']
IOER.dropna(inplace= True)
IOER.plot.line(x='date', y='value')

plt.title ('Interest Rate on Excess Reserves')
plt.xlabel ('Date')
plt.ylabel ('Rate')
plt.axhline(0, 0, 1, label='0')

NameError: name 'fred' is not defined

In [11]:
 """Bank Prime Loan Rate (MPRIME)
 Existing Home Sales (EXHOSLUSM495S)
 Monthly Supply of Houses in the United States (MSACSR)
 Real Gross Domestic Product (GDPC1)
 10-Year Treasury Constant Maturity Rate (DGS10)"""

 #bank exposures by sector 

'Bank Prime Loan Rate (MPRIME)\nExisting Home Sales (EXHOSLUSM495S)\nMonthly Supply of Houses in the United States (MSACSR)\nReal Gross Domestic Product (GDPC1)\n10-Year Treasury Constant Maturity Rate (DGS10)'